In [1]:
import sys
import numpy as np
import pandas as pd

/tmp/ipykernel_128081/1119440127.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
sys.path.append("../") # go to parent dir

In [3]:
from Logs_to_df import Logs_to_df

In [4]:
ldf = Logs_to_df("../../net_samples/1", output_type='full')

Platform server - analysis output_type full


rm: cannot remove '../../net_samples/1/dfs/*': No such file or directory


In [5]:
ldf.output_type="full"
ldf.worker_log_to_df_ND()

In [6]:
wdf = ldf.worker_df

In [7]:
full_names = np.array(ldf.worker_df['fqdn'])

In [8]:
def collapse_similar_funcs(wdf):
    #chameleon.CPU.0-0 -> chamleon 
    def rename_fqdn(fname):
        return fname.split('.')[0]

    wdf['fqdn'] = wdf['fqdn'].apply(rename_fqdn)
    return wdf

In [9]:
collapse_similar_funcs(wdf)

,exec,network,char,time,disk,cpu,mem,fn_start,fn_end,fqdn,tid
0,0.011425,7628.0,cold,0.751738,0.0,0.0,38662144.0,2024-02-21 10:29:28.675719032,2024-02-21 10:29:28.687144032,chameleon,1
1,0.022578,6554.0,warm,0.022578,0.0,0.0,38924288.0,2024-02-21 10:29:28.705401608,2024-02-21 10:29:28.727979608,chameleon,2
2,0.020055,6556.0,warm,0.020055,0.0,0.0,39055360.0,2024-02-21 10:29:28.743875104,2024-02-21 10:29:28.763930104,chameleon,3
3,0.017220,6549.0,warm,0.017220,0.0,0.0,39186432.0,2024-02-21 10:29:28.779062078,2024-02-21 10:29:28.796282078,chameleon,4
4,0.042363,6554.0,warm,0.042363,0.0,0.0,41676800.0,2024-02-21 10:29:28.811561748,2024-02-21 10:29:28.853924748,chameleon,5
...,...,...,...,...,...,...,...,...,...,...,...
215,1.979680,15406.0,warm,1.979680,0.0,0.0,355848192.0,2024-02-21 10:32:52.097842650,2024-02-21 10:32:54.077522650,cnn_image_classification,216
216,1.842575,13229.0,warm,1.842575,0.0,0.0,370651136.0,2024-02-21 10:32:54.095462142,2024-02-21 10:32:55.938037142,cnn_image_classification,217
217,1.881533,12766.0,warm,1.881533,0.0,0.0,384143360.0,2024-02-21 10:32:55.955618748,2024-02-21 10:32:57.837151748,cnn_image_classification,218
218,1.963585,14879.0,warm,1.963585,0.0,0.0,399564800.0,2024-02-21 10:32:57.855324099,2024-02-21 10:32:59.818909099,cnn_image_classification,219


In [10]:
ldf.M

2

In [11]:
ldf.princip_list

['cnn_image_classification', 'chameleon']

In [12]:
from disaggregation import core

In [13]:
cor = core.Core(ldf)

In [14]:
cor.summarize_trace()

array([110., 110.])

In [15]:
ts = cor.get_start_ts()
t1 = ts + pd.Timedelta(seconds=1)
t2 = t1 + pd.Timedelta(seconds=1)
t3 = t2 + pd.Timedelta(seconds=1)
t = t1 
cor.get_C_row(t, 1)

array([0.      , 0.226429])

In [16]:
def get_all_resource_vec(t, delta):
    out = [] 
    dimensions = ['exec','network', 'mem']
    for kind in dimensions:
        one_feat = cor.get_resource_row(t, delta, kind)
        out.append(one_feat)
    return out 

In [17]:
R1 = cor.get_R_row(t1, 1)

In [18]:
R2 = cor.get_R_row(t2, 1)
R3 = cor.get_R_row(t3, 1)

In [19]:
R1.shape

(2, 1, 2)

In [20]:
R1[1]

array([[    0., 73174.]])

In [21]:
np.vstack([R1, R2])

array([[[0.00000e+00, 2.26429e-01]],

       [[0.00000e+00, 7.31740e+04]],

       [[0.00000e+00, 2.30997e-01]],

       [[0.00000e+00, 7.26100e+04]]])

In [22]:
np.hstack([R1, R2, R3])

array([[[0.00000e+00, 2.26429e-01],
        [0.00000e+00, 2.30997e-01],
        [0.00000e+00, 2.29730e-01]],

       [[0.00000e+00, 7.31740e+04],
        [0.00000e+00, 7.26100e+04],
        [0.00000e+00, 7.26200e+04]]])

In [24]:
c.shape

NameError: name 'c' is not defined

In [25]:
np.reshape(c,(2,1))

NameError: name 'c' is not defined

In [26]:
C = cor.build_A_matrix(ts, 10, 1)

In [27]:
C.shape

(10, 2)

In [28]:
R = np.concatenate((R1,R2,R3),axis=1)

In [29]:
R[0][0]

array([0.      , 0.226429])

In [30]:
R = cor.build_A_matrix(ts, 10, 1, "R")

In [31]:
W = np.random.rand((10))

In [32]:
W

array([0.91223799, 0.30507014, 0.33775991, 0.76068282, 0.96808178,
       0.68412384, 0.93305509, 0.4856037 , 0.68553665, 0.30239815])

In [ ]:
import cvxpy as cp

In [ ]:
M = 2
x = cp.Variable((M,2))

In [ ]:
cost = cp.sum_squares(R @ x-W)

In [ ]:
R.shape

In [ ]:
import scipy
import scipy.optimize

In [ ]:
scipy.optimize.lsq_linear(

In [33]:
import sklearn

In [34]:
from sklearn import linear_model

In [35]:
reg = linear_model.LinearRegression()

In [36]:
reg.fit(R, W)

ValueError: Found array with dim 3. LinearRegression expected <= 2.